In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_67373/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 06:43:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/17 06:43:14 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data
data = {
    'power': [2, 1, 0],
    'factor': [1, -4, 2]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)



df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Terms")


import pandas as pd

# Sample data
data = {
    'power': [4, 2, 1],
    'factor': [-4, 1, -1]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Terms_1")


   power  factor
0      2       1
1      1      -4
2      0       2
   power  factor
0      4      -4
1      2       1
2      1      -1


In [30]:
query = """
    
    select
        concat(concat_ws('', collect_list(equation)), "=0") as equation
    from (
        select
            factor,
            power,
            concat(factor, 
                case
                    when power = 1 then "X" 
                    when power = 0 then ""
                    else concat("X^", power)
                end
            ) as equation
        from (
            select
                case
                    when factor >= 0 then concat("+", factor) 
                    else factor
                end as factor,
                power
            from Terms_1 t
        )
        order by power desc
    )
"""

In [31]:
spark.sql(query).show()

+---------------+
|       equation|
+---------------+
|-4X^4+1X^2-1X=0|
+---------------+

